In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym 
import math
import time

In [2]:
env = gym.make('CartPole-v0')
NUM_ACTIONS = env.action_space.n

In [3]:
TIME_STEP = 200

NUM_BUCKET = [5,5,10,12]
EPS = 0.3

STATE_BOUND = np.zeros([4,2])
for i in range(4):
    STATE_BOUND[i] = np.array([env.observation_space.low[i], env.observation_space.high[i]])
STATE_BOUND[1] = np.array([-0.5, 0.5])
STATE_BOUND[3] = np.array([-math.radians(50), math.radians(50)])

In [4]:
def select_action(state,q,eps):
    if np.random.uniform(low=0,high=1.0)<eps:
        return env.action_space.sample()
    else :
        return np.argmax(q[state])

In [50]:
def state_norm(obv):
    state = np.zeros([4,])
    state = obv 
    for i in range(4):
        if obv[i]<=STATE_BOUND[i][0]:
            state[i] = STATE_BOUND[i][0]
        elif obv[i]>=STATE_BOUND[i][1]:
            state[i] = STATE_BOUND[i][1]
    #state = state.astype(int)
    return totuple(state)

In [51]:
def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

In [62]:
def episode(param,dis=0,eps=0):
    obv = env.reset()
    state_in = state_norm(obv)
    total = 0
    ac = 0
    for t in range(250):
        if dis :
            env.render()
        dc = np.matmul(state_in,param)
        #print(t," ",state_in," ",dc)
        if np.random.uniform(0,1)<eps:
            ac = env.action_space.sample()
        else :
            if dc>0:
                ac = 1 
            else:
                ac = 0
        obv,rew,done,_ = env.step(ac)
        #print(obv)
        total = total + 1
        state = state_norm(obv)
        #print("state ",state)
        state_in = state
        if done:
            if dis :
                print("episode ended with ",t+1," steps")
            break
    return total

In [63]:
def v_update(pvel,pbest,gbest,param,w=.75,c1=.75,c2=.75):
    return w*pvel + c1*np.random.uniform(0,.1)*(pbest-param) + c2*np.random.uniform(0,.1)*(gbest-param)

In [64]:
def PSO(nparams=20,nIter=500):
    params = []
    best_params = []
    pbests = []
    vel = []
    gbest = np.zeros([4,])
    gbest_r = 0
    for i in range(nparams):
        params.append(np.random.uniform(-5,5,4))
        best_params.append(np.array([0,0,0,0]))
        pbests.append(0)
        vel.append(np.random.uniform(0,0,4))
    for ep in range(nIter):
        for i in range(nparams):
            c_rew = episode(params[i])
            if c_rew > pbests[i]:
                pbests[i] = c_rew
                best_params[i] = params[i]
        gbest = best_params[np.argmax(np.array(pbests))]
        gbest_r = np.max(np.array(pbests))
        
        if (ep+1)%50==0:
            print("ep = ",ep+1," time : ",gbest_r)
        
        for i in range(nparams):
            vel[i] = v_update(vel[i],best_params[i],gbest,params[i])
            params[i] = params[i] + vel[i]
    return params,best_params,gbest

In [66]:
params,best_params,gbest = PSO()

ep =  50  time :  200
ep =  100  time :  200
ep =  150  time :  200
ep =  200  time :  200
ep =  250  time :  200
ep =  300  time :  200
ep =  350  time :  200
ep =  400  time :  200
ep =  450  time :  200
ep =  500  time :  200


In [74]:
for ep in range(10):
    episode(gbest,1,0)
env.render(close=True)

episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps
episode ended with  200  steps


In [72]:
np.save('CartPole_PSO_gbest.npy',gbest)